In [1]:
import csv
import time
import traceback

import pandas
from ib_insync import *
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import traceback
from copy import deepcopy

from kiteconnect import KiteConnect

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
warnings.filterwarnings('ignore')
import math
import redis
import six
import sys
import time
import json
import struct
import logging
import threading
from datetime import datetime, timedelta
from twisted.internet import reactor, ssl
from twisted.python import log as twisted_log
from twisted.internet.protocol import ReconnectingClientFactory
from autobahn.twisted.websocket import WebSocketClientProtocol, \
    WebSocketClientFactory, connectWS

# from .__version__ import __version__, __title__


In [2]:
def generateAccess():
    global access_token, key_secret, kite, instrument_df, tickers, quantity
    access_token = 'GXOdTC4l6kV6mikGkoLbZcsoIxBZ90MX'
    key_secret = 'j2offcolhvyskf8r'
    kite = KiteConnect(api_key=key_secret)
    kite.set_access_token(access_token)
def instrumentLookup(symbol):
    global access_token, key_secret, kite, instrument_df, tickers, quantity
    try:
        return instrument_df[instrument_df.tradingsymbol == symbol].instrument_token.values[0]
    except:
        return -1


def getInstruments():
    global access_token, key_secret, kite, instrument_df, tickers, quantity
    while True:
        try:
            instrument_dump = kite.instruments()
            instrument_df = pd.DataFrame(instrument_dump)
            break
        except Exception as e:
            print(str(e))
            continue

In [5]:
from kiteconnect import KiteConnect
generateAccess()
getInstruments()
instrument_df.head()
# print(instrument_df['tradingsymbol'].values)

['EURINR23DEC84.5CE' 'EURINR23DEC84.5PE' 'EURINR23DEC84.75CE' ...
 'ZUARIIND' 'ZYDUSLIFE' 'ZYDUSWELL']


In [10]:
def get_atm_nifty(ltp):
    nifty_step_size = 50
    mod = math.fmod(ltp, nifty_step_size)
    if mod >= 25:
        atm_value = ltp - mod + nifty_step_size
    else:
        atm_value = ltp - mod
    #print('Nifty ATM Value: ', int(atm_value))
    return int(atm_value)

def get_atm_bank_nifty(ltp):
    bnf_step_size = 100
    mod = math.fmod(ltp, bnf_step_size)
    if mod > 50:
        atm_value = int((ltp//100)*100)+100
    else:
        atm_value = int(ltp//100)*100
    return atm_value
    

def fetch_ltp(kite, name):
    '''
    Function takes kite object, name of exchange and name of instruments
    and returns the ltp
    '''
    instrument_name = str(instrumentLookup(name))
    
    ltp = kite.ltp(instrument_name)[instrument_name]['last_price']
    return ltp


In [12]:
print(get_atm_nifty(fetch_ltp(kite, 'NIFTY 50')))
print(fetch_ltp(kite, 'NIFTY 50'))
# print(kite.ltp("256265")["256265"]['last_price'])

19750
19727.05


In [13]:
today = datetime.now().strftime("%Y-%m-%d")
type(today)

str

In [14]:
from datetime import datetime, timedelta
def month00():
    today = datetime.now().strftime("%Y-%m-%d")
    months_representation = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN', 7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    year = str(today.year)
    # print(year[-2:])
    return (year[-2:] + months_representation[today.month])

def month01():
    today = datetime.now().strftime("%Y-%m-%d")
    months_representation = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN', 7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    year = str(today.year)
    # print(year[-2:])
    return (year[-2:] + months_representation[(today.month+1)%12])



In [7]:
print(get_tokendate_next())


23OCT


In [96]:
#ITM CE
#50 below ATM
ltp_data = kite.ltp("256265")
ltp = ltp_data["256265"]['last_price']
itm_ce = int(get_atm_nifty(ltp)) - 50

exchange = "NFO"
tradingsymbol = "NIFTY" + month00() + str(itm_ce) + "CE" 
print(tradingsymbol)
try:
    instruments = kite.ltp(exchange + ":" + tradingsymbol)
    if instruments:
        instrument_token_ce = instruments[exchange + ":" + tradingsymbol]["instrument_token"]
        print("Instrument Token:", instrument_token_ce)
    else:
        print("Instrument not found.")
except Exception as e:
    print("Error:", str(e))


NIFTY23SEP19650CE
Instrument Token: 21982466


In [10]:
#ITM PE
#50 above ATM
ltp_data = kite.ltp("256265")
ltp = ltp_data["256265"]['last_price']
itm_ce = int(get_atm_nifty(ltp)) + 50

exchange = "NFO"
tradingsymbol = "NIFTY" + get_tokendate_next() + str(itm_ce) + "PE" 
print(tradingsymbol)
try:
    instruments = kite.ltp(exchange + ":" + tradingsymbol)
    if instruments:
        instrument_token_ce = instruments[exchange + ":" + tradingsymbol]["instrument_token"]
        print("Instrument Token:", instrument_token_ce)
    else:
        print("Instrument not found.")
except Exception as e:
    print("Error:", str(e))


NIFTY23OCT19700PE
Instrument Token: 17231106


In [23]:
instrumentLookup('NIFTY23SEP19500PE')

21977346

In [103]:
def week00():    
    all_expiry_dates = [str(x) for x in sorted(list(set(instrument_df[(instrument_df['exchange'] == 'NFO') & (instrument_df['name'] == 'NIFTY')]['expiry'].values)))]
    x = all_expiry_dates[0]
    y = all_expiry_dates[1]
    if y[5:7] != x[5:7]:
        return month00()
    week = x[-2:]
    year = x[2:4]
    today = datetime.now().strftime("%Y-%m-%d")
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    month = today.month
    month = str(month)
    if month == '10':
        month = 'O'
    if month == '11':
        month = 'N'
    if month == '12':
        month = 'D'
    return (str(year)+month+str(week))
# print(all_expiry_dates[0][-2:])

def week01():    
    all_expiry_dates = [str(x) for x in sorted(list(set(instrument_df[(instrument_df['exchange'] == 'NFO') & (instrument_df['name'] == 'NIFTY')]['expiry'].values)))]
    x = all_expiry_dates[1]
    y = all_expiry_dates[2]
    if y[5:7] != x[5:7]:
        return month00()
    week = x[-2:]
    year = x[2:4]
    today = datetime.now().strftime("%Y-%m-%d")
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    month = today.month
    month = str(month)
    if month == '10':
        month = 'O'
    if month == '11':
        month = 'N'
    if month == '12':
        month = 'D'
    return (str(year)+month+str(week))
# print(all_expiry_dates[0][-2:])



In [90]:
today = datetime.now().strftime("%Y-%m-%d")
time_format = "%Y-%m-%d"
today = datetime.strptime(today, time_format)
month = today.month
month = str(month)
if month == '10':
    month = 'O'
if month == '11':
    month = 'N'
if month == '12':
    month = 'D'
print(month)

9


In [15]:
def month00():
    today = datetime.now().strftime("%Y-%m-%d")
    months_representation = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN', 7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    year = str(today.year)
    # print(year[-2:])
    return (year[-2:] + months_representation[today.month])

def month01():
    today = datetime.now().strftime("%Y-%m-%d")
    months_representation = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN', 7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    year = str(today.year)
    # print(year[-2:])
    return (year[-2:] + months_representation[(today.month+1)%12])

def week00_nf():    
    all_expiry_dates = [str(x) for x in sorted(list(set(instrument_df[(instrument_df['exchange'] == 'NFO') & (instrument_df['name'] == 'NIFTY')]['expiry'].values)))]
    x = all_expiry_dates[0]
    y = all_expiry_dates[1]
    if y[5:7] != x[5:7]:
        return month00()
    week = x[-2:]
    year = x[2:4]
    today = datetime.now().strftime("%Y-%m-%d")
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    month = today.month
    month = str(month)
    if month == '10':
        month = 'O'
    if month == '11':
        month = 'N'
    if month == '12':
        month = 'D'
    return (str(year)+month+str(week))
# print(all_expiry_dates[0][-2:])

def week01_nf():    
    all_expiry_dates = [str(x) for x in sorted(list(set(instrument_df[(instrument_df['exchange'] == 'NFO') & (instrument_df['name'] == 'NIFTY')]['expiry'].values)))]
    x = all_expiry_dates[1]
    y = all_expiry_dates[2]
    if y[5:7] != x[5:7]:
        return month00()
    week = x[-2:]
    year = x[2:4]
    today = datetime.now().strftime("%Y-%m-%d")
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    month = today.month
    month = str(month)
    if month == '10':
        month = 'O'
    if month == '11':
        month = 'N'
    if month == '12':
        month = 'D'
    return (str(year)+month+str(week))
def week00_bn():    
    all_expiry_dates = [str(x) for x in sorted(list(set(instrument_df[(instrument_df['exchange'] == 'NFO') & (instrument_df['name'] == 'BANKNIFTY')]['expiry'].values)))]
    x = all_expiry_dates[0]
    y = all_expiry_dates[1]
    if y[5:7] != x[5:7]:
        return month00()
    week = x[-2:]
    year = x[2:4]
    today = datetime.now().strftime("%Y-%m-%d")
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    month = today.month
    month = str(month)
    if month == '10':
        month = 'O'
    if month == '11':
        month = 'N'
    if month == '12':
        month = 'D'
    return (str(year)+month+str(week))
# print(all_expiry_dates[0][-2:])

def week01_bn():    
    all_expiry_dates = [str(x) for x in sorted(list(set(instrument_df[(instrument_df['exchange'] == 'NFO') & (instrument_df['name'] == 'BANKNIFTY')]['expiry'].values)))]
    x = all_expiry_dates[1]
    y = all_expiry_dates[2]
    if y[5:7] != x[5:7]:
        return month00()
    week = x[-2:]
    year = x[2:4]
    today = datetime.now().strftime("%Y-%m-%d")
    time_format = "%Y-%m-%d"
    today = datetime.strptime(today, time_format)
    month = today.month
    month = str(month)
    if month == '10':
        month = 'O'
    if month == '11':
        month = 'N'
    if month == '12':
        month = 'D'
    return (str(year)+month+str(week))

In [16]:
#ITM CE
#50 below ATM
ltp_data = kite.ltp("256265")
ltp = ltp_data["256265"]['last_price']
itm_ce = int(get_atm_nifty(ltp)) - 50

exchange = "NFO"
tradingsymbol = "NIFTY" + week00_nf() + str(itm_ce) + "CE" 
print(tradingsymbol)
try:
    instruments = kite.ltp(exchange + ":" + tradingsymbol)
    if instruments:
        instrument_token_ce = instruments[exchange + ":" + tradingsymbol]["instrument_token"]
        print("Instrument Token:", instrument_token_ce)
    else:
        print("Instrument not found.")
except Exception as e:
    print("Error:", str(e))


NIFTY2390719700CE
Instrument Token: 11529474


In [52]:
token_list = []

In [20]:
listo = []

In [27]:
atm_ce_nf = int(get_atm_nifty(fetch_ltp(kite, 'NIFTY 50')))-500
print(atm_ce_nf)
for i in range(1, 10):
    itm_ce = atm_ce_nf + (50*i)
    listo.append(itm_ce)
    exchange = "NFO"
    tradingsymbol = "NIFTY" + week01_nf() + str(itm_ce) + "CE" 
    print(tradingsymbol)
    try:
        instrument_token_ce_nf=instrumentLookup(tradingsymbol)
        token_list.append(instrument_token_ce_nf)
    except Exception as e:
        print("Error:", str(e))

print(token_list)   

# print(listo)

19250
NIFTY2391419300CE
NIFTY2391419350CE
NIFTY2391419400CE
NIFTY2391419450CE
NIFTY2391419500CE
NIFTY2391419550CE
NIFTY2391419600CE
NIFTY2391419650CE
NIFTY2391419700CE
[13542658, 13538562, 13539074, 13539586, 13540098, 13540610, 13541122, 13541634, 13542146, 13542658]


In [56]:
tradingsymbol = "NIFTY" + week01_nf() + '19500' + "CE" 
print(tradingsymbol)

NIFTY2391419500CE


In [26]:
instrumentLookup('NIFTY2391419300CE')

13538562

In [46]:
token_list = []

In [64]:
def ITM_ce_nf():
    global token_list
    atm_ce_nf = int(get_atm_nifty(fetch_ltp(kite, 'NIFTY 50')))-500
    # print(atm_ce_nf)
    for i in range(1, 10):
        itm_ce = atm_ce_nf + (50*i)
        listo.append(itm_ce)
        exchange = "NFO"
        tradingsymbol = "NIFTY" + week00_nf() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "NIFTY" + week01_nf() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "NIFTY" + month00() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "NIFTY" + month01() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))

      

In [65]:
def ITM_pe_nf():
    global token_list
    atm_ce_nf = int(get_atm_nifty(fetch_ltp(kite, 'NIFTY 50')))-500
    # print(atm_ce_nf)
    for i in range(1, 10):
        itm_ce = atm_ce_nf + (50*i)
        listo.append(itm_ce)
        exchange = "NFO"
        tradingsymbol = "NIFTY" + week00_nf() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "NIFTY" + week01_nf() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "NIFTY" + month00() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "NIFTY" + month01() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))


In [66]:
def ITM_ce_bn():
    global token_list
    atm_ce_bn = int(get_atm_nifty(fetch_ltp(kite, 'NIFTY BANK')))-1000
    # print(atm_ce_bn)
    for i in range(1, 10):
        itm_ce = atm_ce_bn + (100*i)
        listo.append(itm_ce)
        exchange = "NFO"
        tradingsymbol = "BANKNIFTY" + week00_bn() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "BANKNIFTY" + week01_bn() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "BANKNIFTY" + month00() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "BANKNIFTY" + month01() + str(itm_ce) + "CE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))

In [68]:
def ITM_pe_bn():
    global token_list
    atm_ce_bn = int(get_atm_nifty(fetch_ltp(kite, 'NIFTY BANK')))-1000
    # print(atm_ce_bn)
    for i in range(1, 10):
        itm_ce = atm_ce_bn + (100*i)
        listo.append(itm_ce)
        exchange = "NFO"
        tradingsymbol = "BANKNIFTY" + week00_bn() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "BANKNIFTY" + week01_bn() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "BANKNIFTY" + month00() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))
        tradingsymbol = "BANKNIFTY" + month01() + str(itm_ce) + "PE" 
        try:
            instrument_token_ce_nf=instrumentLookup(tradingsymbol)
            token_list.append(instrument_token_ce_nf)
        except Exception as e:
            print("Error:", str(e))

In [73]:
token_list = []
ITM_pe_nf()
ITM_ce_nf()
ITM_pe_bn()
ITM_ce_bn()

In [74]:
len(token_list)

144

In [ ]:
import subprocess
subprocess.run(["python", "script_redis.py"])

In [119]:
fetch_ltp(kite, 'NSE', 'NIFTY BANK')
instrumentLookup('BANKNIFTY2391444700CE')

-1

In [ ]:
def get_nifty_itm_symbol_updated(df_instruments, ltp, ce_pe, step_size = 50):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    #today = '2023-08-03'
    expiry_day = False
    expiry_date = None
    weekly_expiry, monthly_expiry = False, False
    months_representation = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN', 7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    
    all_expiry_dates = sorted(list(set(df_instruments[(df_instruments['exchange'] == 'NFO') & (df_instruments['name'] == 'NIFTY')]['expiry'].values)))
    dates = [datetime.datetime.strptime(x, '%Y-%m-%d') for x in all_expiry_dates]
    dates = sorted(dates)
    sorted_dates = [datetime.datetime.strftime(x, "%Y-%m-%d") for x in dates]
    #print('Sorted dates: ', sorted_dates)

    # Take first 6 dates and find this months's all expiry dates
    current_month = today.split('-')[1]
    #print('Current month', current_month)

    # Collect all expiry date for this month
    this_month_expiry = []
    for x in sorted_dates[:]:
        #print(x.split('-')[1])
        if x.split('-')[1] == current_month:
            this_month_expiry.append(x)
    #print('This month expiry: ', this_month_expiry)
    
    # Check if today is an expiry day or not
    if today == this_month_expiry[0]:
        expiry_day = True
        #print(f"Today {today} is an expiry day")
    else:
        #print(f"Today {today} is not an expiry day")
        expiry_date = this_month_expiry[0]
        if len(this_month_expiry) == 1:
            monthly_expiry = True
            #print('This is a monthly expiry')


    # this means, today is the monthly expiry date, so select next weekly expiry
    if (expiry_day) and (len(this_month_expiry) == 1):   
        expiry_date = sorted_dates[1]
        #print('Selected Expiry date: ', expiry_date)

    # if today is an expiry day and next expiry is a monthly expiry
    elif ( (expiry_day) and (len(this_month_expiry) == 2) ):
        expiry_date = this_month_expiry[-1]
        #print('Selected Expiry date: ', expiry_date, ' and its a monthly expiry')
        monthly_expiry = True

    # else today is a weekly expiry and just paas the next weekly expiry
    elif ((expiry_day) and (len(this_month_expiry) > 2)):
        expiry_date = this_month_expiry[1]
        #print('Selected Expiry date: ', expiry_date)

    # Else just paas the current expiry date    
    else:
        expiry_date = this_month_expiry[0]
        #print('Selected Expiry date: ', expiry_date)
    
    if monthly_expiry:
        month = int(this_month_expiry[0].split('-')[1])
        #print('Month: ', month)
        nifty_symbol_name = 'NIFTY'+expiry_date.split('-')[0][-2:]+months_representation[month]
        #print('Nifty symbol name: ', nifty_symbol_name)
    else:
        nifty_symbol_name = 'NIFTY'+expiry_date.split('-')[0][-2:]+str(int(expiry_date.split('-')[1]))+expiry_date.split('-')[2]
        #print('Nifty symbol name: ', nifty_symbol_name)

    atm_value = get_atm_nifty(ltp)
    if ce_pe == 'PE':
        itm_value = str(int(atm_value)+step_size)
        nifty_symbol_name = nifty_symbol_name+itm_value+'PE'
    if ce_pe == 'CE':
        itm_value = str(int(atm_value)-step_size)
        nifty_symbol_name = nifty_symbol_name+itm_value+'CE'
    #print('Nifty itm symbol: ', nifty_symbol_name)
    return nifty_symbol_name
